<a href="https://colab.research.google.com/github/dlkt101101/CS431/blob/main/A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## CS431 Data Intensive Distributed Analytics
### Winter 2025 - Assignment 2
---

**Please edit this (text) cell to provide your name and UW student ID number!**
* **Name:** _Darren Lam Kin Teng_
* **ID:** _20977843_

---
#### Overview
For this assignment, you will be using Python and Spark to analyze the [pointwise mutual information (PMI)](http://en.wikipedia.org/wiki/Pointwise_mutual_information) of tokens in the text of Shakespeare's plays.    For this assignment, you will need the same text file (`Shakespeare.txt`) and Python tokenizer module, `simple_tokenize.py`, that you used for the first two assignments.    You will also use the same definition of PMI that was used for [Assignment 1](https://www.student.cs.uwaterloo.ca/~cs451/assignment1-431.html).

We first need to install Apache Spark. Run the next block to download and install Spark. It will take about a minute to finish the instalation.

### For those working on their own machine
If you want to use your own machine instead of colab, you can skip this box and install Spark yourself. Make sure to install findspark too! Put Spark in whichever directory makes the most sense for you. Don't forget to set SPARK_HOME appropriately

(I won't be able to help with this)

In [1]:
!apt-get update -qq > /dev/null
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!curl -Os https://student.cs.uwaterloo.ca/~cs451/spark/spark-3.4.3-bin-hadoop3.tgz
!tar xzf spark-3.4.3-bin-hadoop3.tgz
!pip install -q findspark

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


## Google Colab Only
The following box will set the environment variables to the right spots on Google colab. If using your own machine, you should have set these elsewhere already so you can skip this box

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.3-bin-hadoop3"

To use Spark from within a Python program, it is first necessary to tell the Python interpreter where the Spark installation is located. The code in the following cell tells Python how to find this Spark installation. This code creates SparkContext (sc) for you. Do not change this block.

In [3]:
import findspark
findspark.init()

from pyspark import SparkContext, SparkConf

spark_conf = SparkConf()\
  .setAppName("YourTest")\
  .setMaster("local[*]")

sc = SparkContext.getOrCreate(spark_conf)

Once Python knows where Spark is located, you can create a `SparkContext`.   All Spark commands must run within an active `SparkContext`.   The code below will create a `SparkContext`, and store a reference to the context in the variable `sc`.
The `appName` parameter assigns a name of your choosing to the Spark jobs that are created in this context - this is useful mostly for debugging.   The `master` parameter indicates that Spark jobs will run in local mode, using two threads.   This means that your Spark jobs are not really running on a cluster, and are instead running in a single process on the local machine.   You program Spark jobs the same way whether they run in local mode or on a cluster - the main difference between local and cluster modes is, of course, performance.

Next, let's test that your `SparkContext` has been set up properly by running some simple test code.   This code uses a single Spark job to estimate the value of Euler's number $e$. One way to calculate $e$ is to use the following serries by Jacob Bernoulli:

$p_n = 1 - \frac{1}{1!} + \frac{1}{2!} - \frac{1}{3!} + \cdots + \frac{(-1)^n}{n!} = \sum_{k = 0}^n \frac{(-1)^k}{k!}$

As n tends to infinity, $p_n$ approaches $1/e$.

In the following code,  `parallelize()` and `map()` are Spark *transformations*, and `reduce()` is a Spark *action*.   Study the code in the cell below, then go ahead and run it.   It should take several seconds, since a Spark job is being created and executed, and should print an estimate of $e$ when it finishes.   


In [4]:
import math

n= 10000
inverse_e = sc.parallelize(range(0, n)).map(lambda x: ((-1)**x) * (1 / math.factorial(x))).reduce(lambda x,y:x+y)
e = 1 / inverse_e
print("e = ", e)

e =  2.718281828459044


---
#### Question 1  (4/30 marks):

In the following cell, briefly explain how the example works.   What is the Spark job doing, and how is it used to estimate the value of $e$? How accurate is our estimate?

#### Your answer to Question 1:
The spark job first initializes a RDD from 0 to n-1, these are the x values. The map job evaluates the function: $\frac{(-1)^x}{x!}$. Finally, reduce adds all the values for the summation: $∑_{x=0}^n\frac{(-1)^x}{x!}$.

---
### Important

###### The questions that follow ask you to implement functions whose prototypes are given to you. Do **NOT** change the prototypes of the functions. Do **NOT** write code outside of the functions. All necessary code should be included in the function body (except for import statements). You may declare functions inside of the function body. When marking, we will execute your code by calling the functions from an external program, which is why your code cannot rely on statements running outside functions. Please remove any call to the functions that you may have introduced for test purposes before submitting your notebook.

Note: On this and the following assignment you will typically be told to use Spark, not the Python driver program.
That doesn't mean "do not use Python"! It means you should do as much as possible using RDD transformations and actions, and little-to-nothing in the driver itself.

Example:

`someRDD.collect()[:10]` - this is bad! It's collecting *all* of the data onto the driver when we were only looking for 10  
`someRDD.take(10)` - much better!

Extra examples of what not to do:

`sc.parallelize(myFile.readlines())` - no! Have the cluster load the file using `sc.textFile()` instead!

`newRDD = sc.parallelize(<some computation>.collect())` - no!  
`newRDD = <some computation>` - yes!

---
#### Question 2  (4/30 marks):

Now it is your turn to write some Spark programs. Start with the simple task of counting the number of *distinct* tokens which appear in `Shakespeare.txt`.   You have already written Python code to do this in Assignment 1, but for this assignment we want you to use Spark to solve the same problem.   You should compare the answer you get using Spark with the answer you got from your pure-Python solution from Assignment 1.   Both answers should, of course, be the same.

Run the next block to download the text file (Shakespeare.txt) and the Python tokenizer module (simple_tokenize.py).

### Using own machine

If not on google colab, you might not have wget. If not, you can just ctrl+click those links and save the files in the same directory as your Notebook files. On future assignments you won't need to re-download files you already have (they don't change).

In [5]:
!curl -Os https://student.cs.uwaterloo.ca/~cs451/content/cs431/Shakespeare.txt
!curl -Os https://student.cs.uwaterloo.ca/~cs451/content/cs431/simple_tokenize.py

Your code should use Spark, not the Python driver code, to read `Shakespeare.txt` and do the counting.   The idea is to use Spark to give you a data-parallel alternative to the sequential Python solution you wrote for Assignment 1.

Write your solution for question 2 in the code cell below, by implementing the `count_distinct_tokens` function. It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `count_distinct_tokens` must return the number of distinct tokens.

In [18]:
# A2Q2 BOX
from simple_tokenize import simple_tokenize

# Returns the count of distinct tokens in the `Shakespeare.txt` dataset
def count_distinct_tokens(filename="Shakespeare.txt"):
    # your solution to Question 2 here
    RDD = sc.textFile(filename)
    unique = RDD.flatMap(simple_tokenize).distinct().count()
    return unique

In [48]:
data = ["hello world", "how are you", "spark is great"]
rdd = sc.textFile(data)

In [51]:
rdd.collect()

['hello world', 'how are you', 'spark is great']

In [19]:
count_distinct_tokens()

25975

Since the input file is the same as it was in A1 you should get the same ~26K as you got then.

---
#### Question 3  (4/30 marks):

Next, write a Spark program that will count the number of distinct token pairs in `Shakespeare.txt`, as you did in Assignment 1. Again, ensure that the answer that you get using Spark matches the answer you got in the first assignment.

Write your solution for question 3 by implementing the `count_distinct_pairs` function in the code cell below.   It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `count_distinct_pairs` must return the number of distinct token pairs.



In [121]:
# A2Q3 BOX
from simple_tokenize import simple_tokenize
from itertools import permutations

def getPairs(line):
  """
  getPairs takes in a text file and returns all pairs of tokens per line
  """
  lop = []
  lop.extend((x,y) for x,y in list(permutations(simple_tokenize(line),2)) if x != y)
  return lop


# Returns the count of distinct pairs in the `Shakespeare.txt` dataset
def count_distinct_pairs(filename="Shakespeare.txt"):
  RDD = sc.textFile(filename) # this creates an RDD. (it looks like a list of strings but each line is distributed)
  retrieve_pair_counts = RDD.flatMap(getPairs).distinct().count()
  return retrieve_pair_counts

In [123]:
count_distinct_pairs()

1969760

Again, you should be getting just shy of 2M here, the same value you got on A1.

---
#### Question 4  (6/30 marks):

Next, write Spark code that will calculate the probability $p(x)$ (as defined in Assignment 1) for every distinct token $x$ in `Shakespeare.txt`.   Your function should return the 50 highest-probability tokens, and their counts and probabilities.

Make sure that your solution calculates probabilities and identifies the 50 highest-probability tokens in a data-parallel fashion, using Spark transformations and actions.   Only the 50 highest-probability tokens (and their counts and probabilities) should be returned by Spark to your driver code.

Write your solution for question 4 by implementing the `top_50_tokens_probabilities` function in the code cell below. It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `top_50_tokens_probabilities` must return a list of 50 (probability, count, token) tuples, ordered by probability, that is, the list returned by the function should be of the form: `[(proba1, count1, token1), (proba2, count2, token2), ..., (proba50, count50, token50)]`

In [107]:
# A2Q4 BOX
from simple_tokenize import simple_tokenize

# Returns a list of the top 50 (probability, count, token) tuples, ordered by probability
def top_50_tokens_probabilities(filename="Shakespeare.txt"):
    # your solution to Question 4 here
    RDD = sc.textFile(filename)
    # token_count = RDD.flatMap(simple_tokenize).map(lambda x: (x,1)).reduceByKey(lambda x,y:x+y)
    line_count = RDD.count() # number of lines in the file
    tokens = RDD.map(simple_tokenize).filter(lambda tokens: len(tokens) > 0)
    unordered_result = (
        tokens.flatMap(lambda tokens: [(token,1) for token in set(tokens)])
        .reduceByKey(lambda x,y: x+y)
        .map(lambda x: (round(100*(x[1]/line_count), 2), x[1], x[0]))
    )

    ordered_prob_tokens = unordered_result.takeOrdered(50, key= lambda x:-x[0])
    return ordered_prob_tokens

Hint: "and" is the most frequent token, and "the" is close.

---
#### Question 5  (6/30 marks):

Next, write the code for the function `PMI` that will take a positive integer threshold $T$ as an argument, and then return all token pairs that co-occur at least $T$ times in `Shakespeare.txt`.   For each such pair $(x,y)$, the function should also return PMI$(x,y)$, the co-occurrence count of the pair, the number of times $x$ appears, and the number of times $y$ appears. You can compare the results produced by this code with the results of Two-Token queries (from Assignment 1) for consistency.

Write your solution for question 5 by implementing the function `PMI` in the code cell below. It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `PMI` should return a list of ((token1, token2), pmi, co-occurrence_count, token1_count, token2_count) tuples, that is, the list returned by the function should be of the form: `[((token1, token2), pmi, cooc_count, token1_count, token2_count), (...), ((other_token1, other_token2), other_pmi, other_cooc_count, other_token1_count, other_token2_count)]`.


As always, calculations should be done in data-parallel fashion by using Spark.

Note: You're using a "pairs" approach here, since the key is a pair of tokens! Also, if converted to a dictionary, this list is able to answer two-token queries from A1!

In [127]:
from pdb import line_prefix
# A2Q5 BOX
from simple_tokenize import simple_tokenize
from math import log
from itertools import permutations, combinations

# Returns a list of tuples with the following format:
# ((token1, token2), pmi, co-occurrence_count, token1_count, token2_count)
def PMI(threshold):
    # your solution to Question 5 here
    RDD = sc.textFile("Shakespeare.txt")
    line_count = RDD.count() # number of lines in the file
    tokens = RDD.map(simple_tokenize).filter(lambda tokens: len(tokens) > 0)
    pair_counts = (
        tokens
        .flatMap(lambda x: [(pair,1) for pair in combinations(x,2) if pair[0] != pair[1]])
        .reduceByKey(lambda x,y: x+y)
        .filter(lambda x: x[1] >= threshold)
    )

    token_count = (
        tokens
        .flatMap(lambda tokens: [(x,1) for x in tokens])
        .reduceByKey(lambda x,y: x+y)
    )

    result = (
        pair_counts
        .flatmap(lambda x: x)
    )

    return token_count.take(2)

PMI(1)

[('by', 3768), ('william', 123)]

In [ ]:
def PMI_pairs_approach(T):
    # Alternative implementation using pairs approach

    # Read and tokenize
    lines = sc.textFile("Shakespeare.txt")

    def tokenize_line(line):
        tokens = re.findall(r'\b[a-zA-Z]+\b', line.lower())
        return tokens

    documents = lines.map(tokenize_line).filter(lambda tokens: len(tokens) > 0)
    total_docs = documents.count()

    # Pairs approach
    def pairs_mapper(tokens):
        pairs = []
        unique_tokens = list(set(tokens))  # Use set to get unique tokens per document

        for i, token1 in enumerate(unique_tokens):
            for token2 in unique_tokens[i+1:]:
                # Create sorted pair to ensure consistency
                pair = tuple(sorted([token1, token2]))
                pairs.append((pair, 1))
        return pairs

    # Get co-occurrence counts
    pair_counts_rdd = documents.flatMap(pairs_mapper).reduceByKey(lambda a, b: a + b)

    # Get individual token frequencies
    def token_mapper(tokens):
        unique_tokens = set(tokens)
        return [(token, 1) for token in unique_tokens]

    token_freq_rdd = documents.flatMap(token_mapper).reduceByKey(lambda a, b: a + b)
    token_freq_dict = dict(token_freq_rdd.collect())
    token_freq_broadcast = sc.broadcast(token_freq_dict)

    # Calculate PMI for pairs meeting threshold
    def calculate_pmi_pair(pair_count):
        pair, count = pair_count
        if count >= T:
            token1, token2 = pair
            freq1 = token_freq_broadcast.value.get(token1, 0)
            freq2 = token_freq_broadcast.value.get(token2, 0)

            P_xy = count / total_docs
            P_x = freq1 / total_docs
            P_y = freq2 / total_docs

            if P_x > 0 and P_y > 0 and P_xy > 0:
                pmi = math.log2(P_xy / (P_x * P_y))
                return (pair, pmi, count, freq1, freq2)
        return None

    results = pair_counts_rdd.map(calculate_pmi_pair).filter(lambda x: x is not None).collect()
    token_freq_broadcast.unpersist()

    return results

---
#### Question 6  (6/30 marks):

Finally, write Spark code for the function `PMI_one_token`, that will take a positive integer threshold $T$ and a sample size $N$ as arguments. For every token $x$ in `Shakespeare.txt`, your code should find all tokens $y$ that co-occur with $x$ at least $T$ times, as well as PMI$(x,y)$ for each such pair, the co-occurrence count of the pair, the number of times $x$ appears, and the number of times $y$ appears.

For each $x$, the output of your program should be equivalent to the output that would be produced by a One-Token query on $x$ (see Assignment 1), with threshold $T$. Rather than producing output for all possible tokens $x$, the function should produce output only for $N$ different $x$'s, chosen uniformly at random from among the $x$'s with a non-empty list of co-occurrences.

If there are fewer than $N$ different $x$'s, then the function should output all of them.  In other words, if $T$ is very large, the function would output nothing.

Write your solution for question 6 by implementing the function `PMI_one_token` in the code cell below. It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `PMI_one_token` should return a list of $N$ tuples of the form `(token, [ list_of_cooccurring_tokens ])`, where each entry in `list_of_cooccurring_tokens` is of the form `((token1, token2), pmi, cooc_count, token1_count, token2_count)`.

For instance, if $N$ = 2 and the randomly selected tokens are "if" and "you", a valid format for the value returned by `PMI_one_token` would be:
```
[('if', [(('if', 'you'), -0.09813466615111513, 1975, 16624, 18237), (('if', 'a'), 0.03856379243802408, 1568, 16624, 10569)]), ('you', [(('you', 'if'), -0.09813466615111513, 1975, 18237, 16624)])]
```

Hint: Sampling must be done at the very last step.
Hint: there is an action that returns a sample subset from an RDD. (`takeSample`)
Note: You should be taking a "stripes" approach here. While you *can* use a pairs approach and then convert to stripes at the end, it's not as efficient.

In [ ]:
# A2Q6 BOX
from simple_tokenize import simple_tokenize
from math import log

# Returns a list of samp_size tuples with the following format:
# (token, [ list_of_cooccurring_tokens ])
# where list_of_cooccurring_tokens is of the form
# [((token1, token2), pmi, cooc_count, token1_count, token2_count), ...]
def PMI_one_token(threshold, samp_size):
    # your solution to Question 6 here
    pass # this is so the box is runnable - you should delete this when writing your solution


---
Don't forget to save your workbook!   When you are finished and you are ready to submit your assignment, download your notebook file (.ipynb) from the hub to your machine, and then follow the submission instructions in the assignment.